# Run Fontanka

Instructions
1. Convert `.hic` to ICE-balanced `.mcool` using script `scripts/hic_to_mcool.sh`
2. Select mask: either binary or hand-picked aggregate
3. Run Fontanka

In [3]:
import cooler
import cooltools
import bioframe as bf
import os
import subprocess
import numpy as np
import pandas as pd
from skimage.filters import threshold_li
import yaml

# Across experiments

In [ ]:
with open("../submit_all_config.yaml", "r") as f:                     
    cfg = yaml.safe_load(f)

experiments = [{"hic_file": s.get("mcool"), 
            "data_name": name, 
            "genome": s["genome"], 
            "res": int(s["res"]), 
            "win": int(s["win"])} for name, s in cfg.get("samples", {}).items()]


# Shared parameters
# resolution = 50000      # 50 kb
# window_size = int(6e6)  # 6 Mb
base_save_dir = "/nfs/turbo/umms-minjilab/sionkim/jet_pred"
angle_leniency_deg = 20
num_cores = 4
angle_leniency_rad = np.radians(angle_leniency_deg)
fountain_threshold = 0 # Require positive correlation


for exp in experiments:
    hic_file = exp["hic_file"]
    data_name = exp["data_name"]
    genome = exp["genome"]
    resolution = exp["res"]
    window_size = exp["win"]
    
    # save_dir = os.path.join(base_save_dir, data_name)
    # os.makedirs(save_dir, exist_ok=True)
    save_dir = base_save_dir

    # Load cooler at desired resolution
    clr = cooler.Cooler(f"{hic_file}::resolutions/{resolution}", mode="r")

    # Rename chromosomes to ensure they start with "chr"
    rename_dict = {
        name: name if name.startswith("chr") else f"chr{name}"
        for name in clr.chromnames
    }

    # apply the renaming in-place
    cooler.rename_chroms(clr, rename_dict)

    chromsizes = bf.fetch_chromsizes(genome)

    try:
        # Not all genomes have centromeres (e.g. ce10)
        cens = bf.fetch_centromeres(genome)

        if cens is None or cens.empty:
            raise ValueError(f"No centromeres found for genome {genome}.")
        
        # Otherwise, use the centromeres to build arms
        arms = bf.make_chromarms(chromsizes, cens)
    except ValueError:
        # Just use the chromsizes if no centromeres are available
        arms = pd.DataFrame({
            "chrom": chromsizes.index,
            "start": 0,
            "end": chromsizes.values
        })

        # Sort the dataframe to exactly match the cooler's chromnames order
        arms["chrom"] = pd.Categorical(arms["chrom"], categories=clr.chromnames, ordered=True)

        arms = arms.sort_values("chrom").reset_index(drop=True)


    # Select only chromosomes present in the cooler
    arms = arms[arms.chrom.isin(clr.chromnames)].reset_index(drop=True)

    # Trim based on cooler chrom sizes (cannot exceed)
    # This is a rare case but can happen
    chromsizes_cooler = pd.Series(dict(clr.chromsizes), name="length")
    arms["end"] = arms.apply(lambda r: min(r.end, chromsizes_cooler[r.chrom]), axis=1)

    # Overwrite the defult assignment of the "name" column
    # with genomic string coordinate
    arms["name"] = arms.apply(lambda x: f"{x.chrom}:{x.start}-{x.end}", axis=1)

    # Compute expected cis contact vector
    cvd = cooltools.expected_cis(clr=clr,
                                 view_df=arms,
                                 nproc=num_cores)

    # Save arms and expected vector for fontanka
    arms_save_path = os.path.join(save_dir, f"FONTANKA_{data_name}.arms.tsv")
    arms.to_csv(arms_save_path, sep="\t", index=False, header=False)

    cvd_save_path = os.path.join(save_dir, f"FONTANKA_{data_name}.expected.tsv")
    cvd.to_csv(cvd_save_path, sep="\t", index=False)

    # Extract snips
    snips_path = os.path.join(save_dir, f"FONTANKA_{data_name}.{resolution}.snips.npy")

    cmd = [
        "conda", "run", "-n", "fontanka", # this is needed to run the command in the fontanka conda env
        "fontanka", "slice-windows",
        f"{hic_file}::resolutions/{resolution}",
        snips_path, # this is the output file (i.e. snips)
        "-W", str(window_size),
        "-p", f"{num_cores}", # number of cores
        "--view", arms_save_path,
        "--expected", cvd_save_path,
    ]

    subprocess.run(cmd, check=True)

    # Apply binary fountain mask
    out_path = os.path.join(save_dir, f"FONTANKA_{data_name}.{resolution}.predicted.fountains.tsv")
    mask_cmd = [
        "conda", "run", "-n", "fontanka",
        "fontanka", "apply-binary-fountain-mask",
        f"{hic_file}::resolutions/{resolution}",
        out_path,
        "-A", str(angle_leniency_rad),
        "-W", str(window_size),
        "-p", str(num_cores),
        "--snips", snips_path,
        "--view", arms_save_path,
    ]
    subprocess.run(mask_cmd, check=True)

    # New: thresholding and dropNA
    results = pd.read_csv(out_path, sep="\t", index_col=0)

    results = results.dropna()

    # We apply the same thresholding scheme as in Fontanka example notebook
    li_threshold = threshold_li(results['FS_peaks'].dropna().values)
    peak_threshold = max(fountain_threshold, li_threshold)
    print(f" Using threshold: {peak_threshold}")

    results_thresholded = results.loc[results["FS_peaks"] > peak_threshold].reset_index(drop=True)    

    # Finally, we apply the two thresholding schemes as suggested for Hi-C-like data
    min_FS_ref = np.nanpercentile(results_thresholded["FS_peaks"], 90)
    th_noise = np.nanpercentile(results_thresholded["Scharr_box"], 75)

    results_thresholded = results_thresholded.loc[results_thresholded["FS_peaks"] > min_FS_ref].reset_index(drop=True)  
    results_thresholded = results_thresholded.loc[results_thresholded["Scharr_box"] > th_noise].reset_index(drop=True)  

    results_thresholded.to_csv(out_path.replace(".tsv", ".thresholded.tsv"), sep="\t")

    print(f"Finished processing {data_name} (genome: {genome})")

INFO:root:creating a Pool of 4 workers
/home/sionkim/miniconda3/envs/jet-env/lib/python3.11/site-packages/cooltools/api/expected.py:351: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = dict(iter(bins.groupby("chrom")[clr_weight_name]))


KeyboardInterrupt: 

In [ ]:
# Define experiments to process
# experiments = [
#     # {
#     #     "hic_file" : "/nfs/turbo/umms-minjilab/downloaded_data/mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10.mcool",
#     #     "data_name" : "mESC_CTCF-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10",
#     #     "genome" : "mm10",
#     # },
#     # {
#     #     "hic_file" : "/nfs/turbo/umms-minjilab/downloaded_data/mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10.mcool",
#     #     "data_name" : "mESC_RAD21-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10",
#     #     "genome" : "mm10",
#     # },
#     # {
#     #     "hic_file" : "/nfs/turbo/umms-minjilab/downloaded_data/mESC_WAPL-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10.mcool",
#     #     "data_name" : "mESC_WAPL-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10",
#     #     "genome" : "mm10",
#     # },
#     # {
#     #     "hic_file" : "/nfs/turbo/umms-minjilab/downloaded_data/mESC_YY1-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10.mcool",
#     #     "data_name" : "mESC_YY1-auxin-3hr_microc_Hsieh-2022_GSE178982_mm10",
#     #     "genome" : "mm10",
#     # },
#     # {
#     #     "hic_file" : "/nfs/turbo/umms-minjilab/downloaded_data/GSE130275_mESC_WT_combined_2.6B.ice.mcool", # micro-C
#     #     "data_name" : "GSE130275_mESC_WT_combined_2.6B",
#     #     "genome" : "mm10",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/Repli-HiC_K562_WT_totalS.ice.mcool",
#     #     "data_name": "Repli-HiC_K562_WT_totalS",
#     #     "genome": "hg19",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/DP-thymocytes_WT_hic_Guo-2022_GSE199059_mm10-remapped.ice.mcool",
#     #     "data_name": "DP-thymocytes_WT_hic_Guo-2022_GSE199059_mm10-remapped",
#     #     "genome": "mm10",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9.ice.mcool",
#     #     "data_name": "splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9",
#     #     "genome": "mm9",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GSE199059_CD69negDPWTR1R2R3R4_merged.ice.mcool",
#     #     "data_name": "GSE199059_CD69negDPWTR1R2R3R4_merged",
#     #     "genome": "mm9",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_insitu-hic_4DNFI1UEG1HD.ice.mcool",
#     #     "data_name": "GM12878_insitu-hic_4DNFI1UEG1HD",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_cohesin-SMC1-RAD21-pooled_chiadrop_Kim-2024_4DNFI9JN3S8M_hg38.ice.mcool",
#     #     "data_name": "GM12878_cohesin-SMC1-RAD21-pooled_chiadrop_Kim-2024_4DNFI9JN3S8M_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_CTCF_chiadrop_Kim-2024_4DNFIERR7BI3_hg38.ice.mcool",
#     #     "data_name": "GM12878_CTCF_chiadrop_Kim-2024_4DNFIERR7BI3_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38.ice.mcool",
#     #     "data_name": "GM12878_RNAPII_chiadrop_Kim-2024_4DNFI3ZH8UYR_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_control_chiapet_Kim-2024_GSE158897-GM19239_hg38.ice.mcool",
#     #     "data_name": "GM12878_control_chiapet_Kim-2024_GSE158897-GM19239_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_CTCF_chiapet_Kim-2024_4DNFIR5BPZ5L_hg38.ice.mcool",
#     #     "data_name": "GM12878_CTCF_chiapet_Kim-2024_4DNFIR5BPZ5L_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_RAD21_chiapet_Kim-2024_4DNFIV9RG6YP_hg38.ice.mcool",
#     #     "data_name": "GM12878_RAD21_chiapet_Kim-2024_4DNFIV9RG6YP_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_RNAPII_chiapet_Kim-2024_4DNFICWBQKM9_hg38.ice.mcool",
#     #     "data_name": "GM12878_RNAPII_chiapet_Kim-2024_4DNFICWBQKM9_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/HCT116_RAD21-auxin-0hr_hic_Rao-2017_4DNFIP71EWXC_hg38.ice.mcool",
#     #     "data_name": "HCT116_RAD21-auxin-0hr_hic_Rao-2017_4DNFIP71EWXC_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/HCT116_RAD21-auxin-6hr_hic_Rao-2017_4DNFILIM6FDL_hg38.ice.mcool",
#     #     "data_name": "HCT116_RAD21-auxin-6hr_hic_Rao-2017_4DNFILIM6FDL_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/HCT116_RAD21-auxin-6hr_intacthic_Guckelberger-2024_ENCFF461RFV_hg38.ice.mcool",
#     #     "data_name": "HCT116_RAD21-auxin-6hr_intacthic_Guckelberger-2024_ENCFF461RFV_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF109GNA_hg38.ice.mcool",
#     #     "data_name": "HCT116_RAD21-auxin-0hr_intacthic_Guckelberger-2024_ENCFF109GNA_hg38",
#     #     "genome": "hg38",
#     # },
#     # { # SUCCESSFULLY GENERATED
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/K562_hic_Rao-2014_4DNFI2R1W3YW_hg38.ice.mcool",
#     #     "data_name": "K562_hic_Rao-2014_4DNFI2R1W3YW_hg38",
#     #     "genome": "hg38",
#     # },
#     # {
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/zebrafish-embryo_sperm_hic_Wike-2021_4DNFI4P145EM_z11.ice.mcool",
#     #     "data_name": "zebrafish-embryo_sperm_hic_Wike-2021_4DNFI4P145EM_z11",
#     #     "genome": "danRer11", 
#     # },
#     # {
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10.ice.mcool",
#     #     "data_name": "c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10",
#     #     "genome": "ce10",
#     # },
#     # {
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/c-elegans-JK05-L3_SMC3-auxin-1hr_hic_Kim-2023_GSE237663_ce10.ice.mcool",
#     #     "data_name": "c-elegans-JK05-L3_SMC3-auxin-1hr_hic_Kim-2023_GSE237663_ce10",
#     #     "genome": "ce10",
#     # },
#     # {
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10.ice.mcool",
#     #     "data_name": "c-elegans-JK06-L3_WAPL-auxin-1hr_hic_Kim-2023_GSE237663_ce10",
#     #     "genome": "ce10",
#     # },
#     # {
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/GM12878_intacthic_ENCFF785BPC.ice.mcool",
#     #     "data_name": "GM12878_intacthic_ENCFF785BPC",
#     #     "genome": "hg38",
#     # },
#     # {
#     #     "hic_file": "/nfs/turbo/umms-minjilab/downloaded_data/K562_intacthic_ENCODE-2023_ENCFF808MAG_hg38.ice.mcool",
#     #     "data_name": "K562_intacthic_ENCODE-2023_ENCFF808MAG_hg38",
#     #     "genome": "hg38",
#     # },
# ]